# Adding oscillations to HHH

In [1]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
output_notebook()

import numpy as np

from resistingrhythm.util import poisson_impulse
from resistingrhythm.util import poisson_oscillation
from resistingrhythm.util import mad
from resistingrhythm.util import mae

from fakespikes.util import kappa
from fakespikes.util import fano

from resistingrhythm.neurons import HHH

Loading BokehJS ...

/Users/type/anaconda/envs/py2/lib/python2.7/site-packages/brian2/core/variables.py:174: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return np.issubdtype(np.bool, self.dtype)


# Shared params

In [2]:
time = 3
N = 1

V_e = 0
V_i = -80e-3

tau_e = 5e-3
tau_i = 20e-3
tau_h = 1

w = 2e-6

# Generate some input....

In [3]:
t = 0.4
stim_number = 10
stim_onset = 0.1
stim_offset = 0.5
stim_rate = 60
dt = 1e-5
seed_stim = 7525
ns_in, ts_in = poisson_impulse(
        t,
        stim_onset,
        stim_offset - stim_onset,
        stim_rate,
        n=stim_number,
        dt=dt,
        seed=seed_stim)

# Generate an oscillation

In [ ]:
N_osc = 40
osc_onset = 0
n_cycles = 20

osc_rate = 2
f = 10

# Generate the rate 
ns_osc, ts_osc = poisson_oscillation(
    time, 
    osc_onset,
    n_cycles,
    osc_rate,
    f,
    phi=0,
    n=N_osc,
    dt = 1e-5
)

In [ ]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts_osc, ns_osc, color="grey")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, time)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Run HHH

In [ ]:
results = HHH(
    time,
    ns_in,
    ts_in,
    ns_osc, # osc
    ts_osc,
    N=N,
    Ca=20e-6,
    Ca_target=20e-6,
    tau_h=tau_h,
    w_in=w,
    tau_in=tau_e,
    V_in=V_e,
    bias_in=0.0e-9,
    w_osc=w,
    tau_osc=tau_e,
    V_osc=V_e,
    sigma=0
)

In [ ]:
results.keys()
results['v_m'].shape

In [ ]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts_in, ns_in, color="grey")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, time)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
p = figure(plot_width=400, plot_height=200)
p.circle(results['ts'], results['ns'], color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, time)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
p = figure(plot_width=400, plot_height=200)
for n in range(N):
    v = results['v_m'][n, :]
    p.line(x=results['times'], y=v, color="black", alpha=0.5)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'V_m (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range = Range1d(0, time)
show(p)

In [ ]:
p = figure(plot_width=400, plot_height=200)
for n in range(N):
    v = results['g_total'][n, :]
    p.line(x=results['times'], y=v, color="black", alpha=0.5)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'g_total (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range = Range1d(0, time)
show(p)

In [ ]:
p = figure(plot_width=400, plot_height=200)
for n in range(N):
    v = results['calcium'][n, :]
    p.line(x=results['times'], y=v, color="black", alpha=0.5)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Ca (moles)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range = Range1d(0, time)
show(p)

In [ ]:
p = figure(plot_width=400, plot_height=200)
for n in range(N):
    v = results['g_K'][n, :]
    p.line(x=results['times'], y=v, color="black", alpha=0.5)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'g_K (siemens)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range = Range1d(0, time)
show(p)

In [ ]:
p = figure(plot_width=400, plot_height=200)
for n in range(N):
    v = results['g_Na'][n, :]
    p.line(x=results['times'], y=v, color="black", alpha=0.5)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'g_Na (siemens)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range = Range1d(0, time)
show(p)